In [1]:
from shutil import copy
from collections import defaultdict
import scipy
import numpy as np
import matplotlib.pyplot as plt

import random
import cv2
import glob
from PIL import Image, ImageEnhance
import PIL.ImageOps

import torch
import torchvision
from torch import nn


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# Theano(th)와 Tensorflow(tf) 모두와 호환이 되는 Keras 모듈을 작성
import keras.backend as K
from keras import regularizers
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from keras.layers import Dense, Dropout, Conv2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator # 데이터 전처리
from keras.callbacks import ModelCheckpoint, CSVLogger
#from keras.optimizers import SGD

from keras_preprocessing.image import array_to_img, img_to_array, load_img

import tensorflow as tf

from deeplab2.model.pixel_encoder import moat
from AdaBelief_tf import AdaBeliefOptimizer

## Data augmentation (한번만 실행)

In [60]:
file_train_path = "/Users/dayeonjung/Desktop/df/catdog/train/"

catdog_list = os.listdir(file_train_path)

catdog_list.remove('.DS_Store')

catdog_list = list(map(int, catdog_list))
catdog_list.sort()
catdog_list = list(map(str, catdog_list))

for i in range(len(catdog_list)):
    file_path = file_train_path + catdog_list[i] + '/'
    x = os.listdir(file_path)
    file_names=[]
    for fn in x:
        if '.jpg' not in fn:
            continue
        else:
            file_names.append(fn)

    for j in range(0, len(file_names)):
        file_name = file_names[j]
        origin_image_path = file_path +  file_name
        x = Image.open(origin_image_path)

        x = img_to_array(x)
        x = x/255 
        

        if x.shape[2]==3:
            grayscaled = tf.image.rgb_to_grayscale(x)
            saturated = tf.image.adjust_saturation(x, 3)
            bright = ImageEnhance.Brightness(array_to_img(x)).enhance(2.0)
        elif x.shape[2]>3:
            grayscaled = tf.image.rgb_to_grayscale(x[:,:,:3])
            saturated = tf.image.adjust_saturation(x[:,:,:3], 3)
            bright = ImageEnhance.Brightness(array_to_img(x[:,:,:3])).enhance(2.0)
        else:
            grayscaled = x
            saturated = x
            bright = x

        array_to_img(grayscaled).save(file_path + 'gray_' + file_name)
        array_to_img(saturated).save(file_path + 'saturated_' + file_name)
        array_to_img(bright).save(file_path + 'bright_' + file_name)
    print(i+1, end = ' ')

print("train done!")

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 train done!


In [2]:
# releases the global state: avoid clutter from old models and layers
K.clear_session()

In [3]:
# 초기 변수 설정
n_classes = 37

### 이미지 크기 설정
img_width, img_height = 256,256


train_data_dir = '/Users/dayeonjung/Desktop/df/catdog/train'
validation_data_dir = '/Users/dayeonjung/Desktop/df/catdog/validation/'
nb_train_samples = 3312*4   # augmentation했으니 4 곱해주기     
nb_validation_samples = 368  
batch_size = 10   

In [4]:
# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)
train_datagen = ImageDataGenerator(
    rescale=1./ 255,       # multiply the data by the value
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.2,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest')   


test_datagen = ImageDataGenerator(rescale=1./ 255)

In [5]:
# flow_from_directory : Numpy Array Iterator 객체 생성
# 인자로 설정해주는 directory의 바로 하위 디렉토리 이름을 레이블이라고 간주, 그 디렉토리 아래의 파일들을 해당 레이블의 이미지들이라고 알아서 추측


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')
    #'categorical' : 멀티-레이블 클래스, 원-핫 인코딩된 형태
    #'sparse' : 멀티-레이블 클래스, 레이블 인코딩된 형태
    #'binary' : 이진 분류 클래스, 0 또는 1인 형태
    
    
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

Found 13248 images belonging to 37 classes.
Found 368 images belonging to 37 classes.


In [7]:
moat1=moat.get_model(
    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])

Metal device set to: Apple M1


In [8]:
moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

In [9]:
from tensorflow.keras.layers import (Input, Dense, concatenate, Conv2D, MaxPooling2D, Flatten)
from tensorflow import keras

moat1.input_shape
moat1.output_shape

{'stage1': (None, 128, 128, 32),
 'res1': (None, 128, 128, 32),
 'stage2': (None, 64, 64, 32),
 'res2': (None, 64, 64, 32),
 'stage3': (None, 32, 32, 64),
 'res3': (None, 32, 32, 64),
 'stage4': (None, 16, 16, 128),
 'res4': (None, 16, 16, 128),
 'stage5': (None, 8, 8, 256),
 'res5': (None, 8, 8, 256)}

In [10]:
y11 = moat1.output["stage1"]
y12 = moat1.output["res1"]
y21 = moat1.output["stage2"]
y22 = moat1.output["res2"]
y31 = moat1.output["stage3"]
y32 = moat1.output["res3"]
y41 = moat1.output["stage4"]
y42 = moat1.output["res4"]
y51 = moat1.output["stage5"]
y52 = moat1.output["res5"]

z1 = y11+y12
z2 = y21+y22
z3 = y31+y32
z4 = y41+y42
z5 = y51+y52

a1=Conv2D(8,(1,1),activation='relu')(z1)
a2=Conv2D(8,(3,3),activation='relu')(a1)
a3=MaxPooling2D(2,2)(a2) 
a4=Conv2D(8,(3,3),activation='relu')(a3)
a5=MaxPooling2D(2,2)(a4) 
a6=Conv2D(8,(3,3),activation='relu')(a5)
a7=MaxPooling2D(2,2)(a6) 
a8=Conv2D(8,(3,3),activation='relu')(a7)
a9=MaxPooling2D(2,2)(a8)
a10=Conv2D(8,(3,3),activation='relu')(a9)
a11=MaxPooling2D(2,2)(a10)
a12=Conv2D(32,(1,1),activation='relu')(a11)
aa1=Flatten(name='flatten_layer1')(a12)

b1=Conv2D(8,(1,1),activation='relu')(z2)
b2=Conv2D(8,(3,3),activation='relu')(b1)
b3=MaxPooling2D(2,2)(b2) 
b4=Conv2D(8,(3,3),activation='relu')(b3)
b5=MaxPooling2D(2,2)(b4) 
b6=Conv2D(8,(3,3),activation='relu')(b5)
b7=MaxPooling2D(2,2)(b6) 
b8=Conv2D(8,(3,3),activation='relu')(b7)
b9=MaxPooling2D(2,2)(b8)
b10=Conv2D(32,(1,1),activation='relu')(b9)
bb1=Flatten(name='flatten_layer2')(b10)

c1=Conv2D(16,(1,1),activation='relu')(z3)
c2=Conv2D(16,(3,3),activation='relu')(c1)
c3=MaxPooling2D(2,2)(c2) 
c4=Conv2D(16,(3,3),activation='relu')(c3)
c5=MaxPooling2D(2,2)(c4) 
c6=Conv2D(16,(3,3),activation='relu')(c5)
c7=MaxPooling2D(2,2)(c6) 
c8=Conv2D(64,(1,1),activation='relu')(c7)
cc1=Flatten(name='flatten_layer3')(c8)

d1=Conv2D(32,(1,1),activation='relu')(z4)
d2=Conv2D(32,(3,3),activation='relu')(d1)
d3=MaxPooling2D(2,2)(d2) 
d4=Conv2D(32,(3,3),activation='relu')(d3)
d5=MaxPooling2D(2,2)(d4) 
d6=Conv2D(128,(1,1),activation='relu')(d5)
dd1=Flatten(name='flatten_layer4')(d6)

e1=Conv2D(64,(1,1),activation='relu')(z5)
e2=Conv2D(64,(3,3),activation='relu')(e1)
e3=MaxPooling2D(2,2)(e2) 
e4=Conv2D(64,(2,2),activation='relu')(e3)
e5=Conv2D(256,(1,1),activation='relu')(e4)
ee1=Flatten(name='flatten_layer5')(e5)

fc1=tf.concat([aa1,bb1,cc1,dd1,ee1],axis=1)
out1=Dense(units=37, activation='softmax')(fc1)

model1 = tf.keras.models.Model(moat1.input, out1)

In [11]:
# model1.load_weights('model1check.hdf5')

opt = AdaBeliefOptimizer(learning_rate=1e-3, beta_1= 0.9, beta_2=0.999, weight_decay= 1e-4,  epsilon=5e-13, rectify=True)
model1.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.



In [ ]:
checkpointer = ModelCheckpoint(filepath='best_model_3class_sept.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('history.log')

history = model1.fit_generator(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,      #  한 epoch에 사용한 스텝 수
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  # 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수
                    epochs=50,                                             # 전체 훈련 데이터셋에 대해 학습 반복 횟수
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

model1.save('model1_trained.h5')

Epoch 1/50


/var/folders/cc/6xhsqlks6mg1kzb03g_1zn7r0000gn/T/ipykernel_8790/1685723023.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model1.fit_generator(train_generator,


  18/1324 [..............................] - ETA: 2:51:58 - loss: 4.0763 - sparse_categorical_accuracy: 0.0222